# Wikipedia Data Ingestion

### Import Libraries

In [1]:
import os
import logging
import json
import time
import urllib.parse
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
# Read csv file with names of all cities by states 
df_cities = pd.read_csv('us_cities_by_state_SEPT.2023.csv')

In [3]:
# Check for duplicates
df_cities[df_cities.page_title.duplicated()]

,state,page_title,url
461,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
462,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
463,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
464,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
465,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"
...,...,...,...
21349,Virginia,"Waynesboro, Virginia","https://en.wikipedia.org/wiki/Waynesboro,_Virg..."
21350,Virginia,"Williamsburg, Virginia","https://en.wikipedia.org/wiki/Williamsburg,_Vi..."
21351,Virginia,"Winchester, Virginia","https://en.wikipedia.org/wiki/Winchester,_Virg..."
21865,Wisconsin,County (United States),https://en.wikipedia.org/wiki/County_(United_S...


> There are duplicate page_titles/urls, so let's drop them

In [4]:
# Fet rid of duplicate state names or URLs
df_cities = df_cities.drop_duplicates('url')
df_cities = df_cities.drop_duplicates('page_title')

In [5]:
df_cities.head()

,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


## Page Info Request

In [6]:
# Use the template provided to request data from the API
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

REQUEST_HEADERS = {
    'User-Agent': '<vaibhav1@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

ARTICLE_TITLES = ['Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat']

PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
}

In [7]:
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    

    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    try:
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

### The API allows upto 50 articles to be requested at once, hence making the process faster

In [ ]:
# Request and save data for future processing
request_size = 50
for i in tqdm(range(len(df_cities)//request_size+1)):
    info = request_pageinfo_per_article(' | '.join(df_cities.page_title[i*request_size:i*request_size+request_size].tolist()))
    with open(f'json_data/id_{i*request_size}_{i*request_size+request_size-1}.json', 'w') as f:
        json.dump(info, f)

In [8]:
# Read individual JSON files and combine into one large dataframe for processing
df_list = []
for file in os.listdir('json_data/'):    
    with open(f'json_data/{file}') as f:
        d = json.load(f)
        df_list.append(pd.DataFrame(d['query']['pages']).T)

df = pd.concat(df_list)
df = df.sort_values(by='title')

### Merge the requested data with exsiting dataframe of city names

In [9]:
# Merge this newly donwloaded data with the dataset of all the cities
df_merged = df[['pageid', 'title', 'lastrevid']].merge(df_cities, how='right', left_on='title', right_on='page_title').drop_duplicates('title')

In [10]:
# Check if any city was missed by the API
missing = set(df_cities.page_title).difference(set(df_merged.title))
missing = list(missing)
assert len(missing) == 0

In [11]:
# Check if any article in the merged dataset does not have a revision ID
df_merged[df_merged.lastrevid.isna()]

,pageid,title,lastrevid,state,page_title,url


We can see that no articles are missing from our API calls and that each item in the merged dataframe has a lastrevid

# ORES Scores

In [12]:
# Using our new dataset, query the ORES API to get article scores
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"


API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "vaibhav1@uw.edu",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}


ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }


ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

# Personal Access Token, key has been removed to prevent misuse
USERNAME = "Vbrox2121"
ACCESS_TOKEN = ""

In [13]:
def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [ ]:
# Create and configure logger to keep track of each API request
logging.basicConfig(filename=f"pipeline.log",
                    format='%(asctime)s %(message)s',
                    filemode='a',
                    level=logging.INFO)
logger = logging.getLogger()

# Use TQDM to get a progress bar
with tqdm(df_merged.lastrevid) as t:
    for id in t:
        score = request_ores_score_per_article(article_revid=id,
                                           email_address="vaibhav1@uw.edu",
                                           access_token=ACCESS_TOKEN)
        # Save each JSON dump into a file for downstream processing
        with open(f'ores_json_data/{id}.json', 'w') as f:
            json.dump(score, f)
        
        logger.info(f'Processed ID {id} with Score {score}')

In [14]:
# Load the JSON data and combine into one dataframe, focusing just on the scores
df_list = []
missing = []
for file in os.listdir('ores_json_data/'):    
    with open(f'ores_json_data/{file}') as f:
        try:
            d = json.load(f)
        except UnicodeDecodeError as e:
            print(f'Error {e} in {file}')
        if 'httpReason' in d.keys():
            missing.append(file)
            print('No record found for', file)
            continue
        df_list.append(pd.DataFrame(d['enwiki']['scores']).T)

df_ores = pd.concat(df_list)

In [15]:
# Ensure that no article was missed
assert missing == []

> Looks like we were able to get ORES scores for all articles!

In [16]:
# Pandas magic to get one clean dataframe
df_pred = df_ores.articlequality.apply(pd.Series).score.apply(pd.Series)
df_pred = df_pred.reset_index()
df_pred['index'] = df_pred['index'].astype(int)

### Merge ORES scores with our previous dataframe

In [17]:
# Merge the previous dataset with city names and the revision id of thier articles along with the ORES scores
df_result = df_merged.merge(df_pred, left_on='lastrevid', right_on='index')

In [18]:
# Take subset of relevant fields to keep the data clean
df_result = df_result[['pageid', 'title', 'lastrevid', 'state', 'page_title', 'prediction']]

## Read Census Data for regions

In [19]:
# Read the excel file which associates states to region and divions
census_states = pd.read_excel('US States by Region - US Census Bureau.xlsx', index_col=[0,1])
census_states = census_states.dropna()
census_states = census_states.reset_index()
census_states.STATE = census_states.STATE.str.replace(' ', '_') 

In [20]:
# We can see that Georgia_(U.S._state) is present in our merged dataframe but not in the census dataset
# This will be lost in the join, and is ignored for now
(set(df_result.state.unique())).difference(set(census_states.STATE))

{'Georgia_(U.S._state)'}

In [27]:
# We can see that 'Connecticut', 'Georgia', 'Nebraska' present in the census data but not in our analysis
# These will be lost in the join, and are ignored for now
for state in (set(census_states.STATE)).difference((set(df_result.state))):
    print(state)

Georgia
Nebraska
Connecticut


In [28]:
# Merge the census data with our dataset
df_result = df_result.merge(census_states, left_on='state', right_on='STATE').drop(['STATE'], axis=1)

## Read Population Data 

In [29]:
# Load population excel file downloaded from https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html
df_pop = pd.read_excel('NST-EST2022-POP.xlsx', header=[2])[1:]
df_pop.head(10)

,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),Unnamed: 3,Unnamed: 4
1,United States,331449520.0,331511512.0,332031554.0,333287557.0
2,Northeast,57609156.0,57448898.0,57259257.0,57040406.0
3,Midwest,68985537.0,68961043.0,68836505.0,68787595.0
4,South,126266262.0,126450613.0,127346029.0,128716192.0
5,West,78588565.0,78650958.0,78589763.0,78743364.0
6,.Alabama,5024356.0,5031362.0,5049846.0,5074296.0
7,.Alaska,733378.0,732923.0,734182.0,733583.0
8,.Arizona,7151507.0,7179943.0,7264877.0,7359197.0
9,.Arkansas,3011555.0,3014195.0,3028122.0,3045637.0
10,.California,39538245.0,39501653.0,39142991.0,39029342.0


> This needs some cleaning up

In [30]:
# since we are using jsons, ' ' character can be changed to '_' often
df_pop['Geographic Area'] = df_pop['Geographic Area'].str.replace('.', '')
df_pop['Geographic Area'] = df_pop['Geographic Area'].str.replace(' ', '_')

> Due to some formatting issues, we can see that parts of the meta data are coming in as data cells

In [31]:
# Checking the tail of the dataset, we can see which rows to delete
df_pop.tail(6)

,Geographic Area,"April 1, 2020 Estimates Base",Population Estimate (as of July 1),Unnamed: 3,Unnamed: 4
58,Puerto_Rico,3285874.0,3281557.0,3262693.0,3221789.0
59,Note:_The_estimates_are_developed_from_a_base_...,NaN,NaN,NaN,NaN
60,Suggested_Citation:,NaN,NaN,NaN,NaN
61,Annual_Estimates_of_the_Resident_Population_fo...,NaN,NaN,NaN,NaN
62,"Source:_US_Census_Bureau,_Population_Division",NaN,NaN,NaN,NaN
63,Release_Date:_December_2022,NaN,NaN,NaN,NaN


In [32]:
df_pop = df_pop[:58]

In [33]:
set(df_pop['Geographic Area']).difference(set(df_result.state))

{'Connecticut',
 'District_of_Columbia',
 'Georgia',
 'Midwest',
 'Nebraska',
 'Northeast',
 'Puerto_Rico',
 'South',
 'United_States',
 'West',
 nan}

> This is a list of region/states that are not included in out analysis but are provided in the population census. We have already noted that some of them were missing from our initial list, whereas others like Puerto Rico are new

In [34]:
df_result = df_result.merge(df_pop[['Geographic Area', 'April 1, 2020 Estimates Base']], left_on='state', right_on='Geographic Area').drop(['Geographic Area', 'page_title'], axis=1)

In [35]:
df_result['regional_devision'] = df_result.apply(lambda x: f'{x.REGION} ({x.DIVISION})', axis=1)
df_result = df_result.drop(['REGION', 'DIVISION'], axis=1)

In [36]:
df_result

,pageid,title,lastrevid,state,prediction,"April 1, 2020 Estimates Base",regional_devision
0,104730,"Abbeville, Alabama",1171163550,Alabama,C,5024356.0,South (East South Central)
1,104761,"Adamsville, Alabama",1177621427,Alabama,C,5024356.0,South (East South Central)
2,105188,"Addison, Alabama",1168359898,Alabama,C,5024356.0,South (East South Central)
3,104726,"Akron, Alabama",1165909508,Alabama,GA,5024356.0,South (East South Central)
4,105109,"Alabaster, Alabama",1179139816,Alabama,C,5024356.0,South (East South Central)
...,...,...,...,...,...,...,...
20976,140221,"Wamsutter, Wyoming",1169591845,Wyoming,GA,576837.0,West (Mountain)
20977,140185,"Wheatland, Wyoming",1176370621,Wyoming,GA,576837.0,West (Mountain)
20978,140245,"Worland, Wyoming",1166347917,Wyoming,GA,576837.0,West (Mountain)
20979,140070,"Wright, Wyoming",1166334449,Wyoming,GA,576837.0,West (Mountain)


In [46]:
# Save results as per schema
df_result.rename(columns={'April 1, 2020 Estimates Base':'population', 
                         'title': 'article_title',
                         'lastrevid': 'revision_id',
                         'prediction': 'article_quality'})[['state', 'regional_devision', 'population',
                                                           'article_title', 'revision_id', 'article_quality']].to_csv('wp_scored_city_articles_by_state.csv')

# Analysis

### Top 10 US states by coverage: The 10 US states with the highest total articles per capita (in descending order) 

In [37]:
df = df_result.copy()
df_top10 = df.groupby('state').count()['title']/df.groupby('state').min()['April 1, 2020 Estimates Base']

df_top10.sort_values(ascending=False)[:10]

state
Vermont          0.000512
North_Dakota     0.000457
Maine            0.000355
South_Dakota     0.000351
Iowa             0.000327
Alaska           0.000203
Pennsylvania     0.000197
Michigan         0.000176
Wyoming          0.000172
New_Hampshire    0.000170
dtype: float64

#### Bottom 10 US states by coverage: The 10 US states with the lowest total articles per capita (in ascending order) .


In [38]:
df = df_result.copy()
df_bottom10 = df.groupby('state').count()['title']/df.groupby('state').min()['April 1, 2020 Estimates Base']

df_bottom10.sort_values(ascending=True)[:10]

state
North_Carolina    0.000005
Nevada            0.000006
California        0.000012
Arizona           0.000013
Virginia          0.000015
Oklahoma          0.000019
Florida           0.000019
Kansas            0.000021
Maryland          0.000025
Wisconsin         0.000032
dtype: float64

#### Top 10 US states by high quality: The 10 US states with the highest high quality articles per capita (in descending order) .


In [39]:
df = df_result.copy()
df['quality_count'] = df.prediction.apply(lambda x: 1 if x == 'FA' or  x == 'GA' else 0)
df_top10 = df.groupby('state').sum()['quality_count']/df.groupby('state').min()['April 1, 2020 Estimates Base']

df_top10.sort_values(ascending=False)[:10]

state
Vermont          0.000070
Wyoming          0.000068
South_Dakota     0.000063
West_Virginia    0.000059
Montana          0.000051
New_Hampshire    0.000046
Pennsylvania     0.000044
Missouri         0.000043
Alaska           0.000042
New_Jersey       0.000041
dtype: float64

#### Bottom 10 US states by high quality: The 10 US states with the lowest high quality articles per capita (in ascending order).


In [40]:
df = df_result.copy()
df['quality_count'] = df.prediction.apply(lambda x: 1 if x == 'FA' or  x == 'GA' else 0)
df_top10 = df.groupby('state').sum()['quality_count']/df.groupby('state').min()['April 1, 2020 Estimates Base']

df_top10.sort_values(ascending=True)[:10]

state
North_Carolina    0.000002
Virginia          0.000002
Nevada            0.000003
Arizona           0.000003
California        0.000004
New_York          0.000005
Florida           0.000006
Maryland          0.000007
Kansas            0.000007
Oklahoma          0.000008
dtype: float64

#### Census divisions by total coverage: A rank ordered list of US census divisions (in descending order) by total articles per capita.


In [41]:
df_top10 = df.groupby('regional_devision').count()['title']/df.groupby('regional_devision').min()['April 1, 2020 Estimates Base']
df_top10.sort_values(ascending=False)

regional_devision
Midwest (West North Central)    0.004591
Northeast (New England)         0.002235
West (Mountain)                 0.002061
West (Pacific)                  0.001778
South (South Atlantic)          0.001325
Midwest (East North Central)    0.000806
South (West South Central)      0.000697
South (East South Central)      0.000516
Northeast (Middle Atlantic)     0.000407
dtype: float64

#### Census divisions by high quality coverage: Rank ordered list of US census divisions (in descending order) by high quality articles per capita.


In [42]:
df_bottom10 = df.groupby('regional_devision').sum()['quality_count']/df.groupby('regional_devision').min()['April 1, 2020 Estimates Base']
df_bottom10.sort_values(ascending=True)

regional_devision
South (East South Central)      0.000107
Northeast (Middle Atlantic)     0.000114
Midwest (East North Central)    0.000122
South (West South Central)      0.000210
Northeast (New England)         0.000350
South (South Atlantic)          0.000438
West (Mountain)                 0.000581
West (Pacific)                  0.000668
Midwest (West North Central)    0.000820
dtype: float64